# Big Data Product: Burglary Protection 

In this scenario, you are a data scientist working with a marketing consultancy. Your client is an insurance company that is developing a highly segmented home insurance product.

Since it is hypothesized that customers who live in an area where burglary is prevalent would be more interested in a new insurance policy, the company would like to find out whether "Burglary" is more frequent in particular areas of England. 

If that is the case the company needs to determine whether these are arcas of affluence, where a premium policy with high benefits could be sold, or one of relative deprivation where a low-cost economic policy with proportionately lower pay-outs would be more appropriate. Furthermore, the company would like to target areas where burglary is known to be increasing The questions to be answered are whether:
1. There are more burglaries in more affluent areas
2. Burglaries are increasing, decreasing, or are stable


### To solve this problem, you will use publicly available data sets that have been prepared for you and placed online. These include (but are not limited to):-

1.	Street Level Crime Data published by the UK Home Office, this dataset contains 19 million data rows giving a crime type, together with their location as a latitude and longitude.
2.	English Indices of Deprivation Data: The English Indices of Deprivation 2010 data set contains the rankings of measures of deprivation within small area level across England. The 32000 localities are ranked from the least to most deprived, scored on seven different dimensions of deprivation.

# Assignment Specifics

1.	Process the given data efficiently using Apache Spark on a cloud Infrastructure as a Service (IaaS) platform. A sample Jupyter Notebook has been provided on Blackboard.
2.	Filter the dataset so that only relevant crimes are included.
3.	Using appropriate techniques, determine whether Burglaries are increasing, decreasing, or are stable.
4.	Using appropriate techniques, determine whether Burglary is more closely associated with areas of high affluence, relative deprivation or neither
5.	Select and prepare no more than four visualizations to support your analytic findings from (3).
6.	Explain the reasoning behind your code so that it is clear what each block is intended to achieve (i.e., appropriately comment the command line).
8.	Assess the two claims given and determine whether they are true, false, or cannot be determined.
9.	Critically assess and report on the advantages, disadvantages, and limitations of the methods used. 
10.	Your submission will be a Jupyter Notebook containing both code (typically Python), and explanatory text (in Markdown format) limited to 2500 words (plus references). 

# 1. Introduction

## The Burglary Crime Analysis task and Approach taken to the problem

##### Crimes in general is a huge problem for society, peace of mind, locality and affects various parts of life for many people.

#### Burglary Crime Introduction


In recent years, there has been a growing interest in understanding the spatial distribution of crime, particularly burglary, across different regions in England. This is important for various stakeholders, including insurance companies, as it helps them develop targeted products and services. For instance, a highly segmented home insurance product can be tailored to areas with high burglary rates to offer better coverage and protection for the residents. To achieve this, we will examine available data sources such as the Office for National Statistics (ONS) and UK Police Crime Data to identify areas with higher burglary prevalence and understand their socio-economic characteristics (Office for National Statistics, 2021; Police UK, 2021). By determining whether these areas are affluent or relatively deprived, we can help the insurance company design appropriate policies that cater to the specific needs and preferences of the customers in those areas.

To accomplish this, present study will leverage available data sources such as the Office for National Statistics (ONS), UK Police Crime Data, and other government sources to obtain accurate and up-to-date crime data. Through a combination of statistical analysis, data visualization, and clustering algorithms, we will deliver actionable insights that will help our client create a targeted and effective home insurance product for the English market.


#### Approach taken
* Using Big data processing tool (PySpark) running in cloud infrastructure IAAS (like AWS ec2) or PAAS (like AWS Glue or Azure Data Factory)
* The analysis is to study burglary crimes happening in United Kingom (UK) between time period Dec-2010 to April-2020 
* Using various statistical & visualization techniques we study trends and assert various claims by Insurance Client.
* How are we doing the analysis?
   1. Use PySpark to process & crunch the data for better handling of raw data (~19MM rows)
   2. Look at various visual lenses to translate the data into more digestable 
   3. Apply statistical methods to help to test various hypothesis & claims


#### Difference beween IAAS and PAAS 
|Index|Topic|IAAS|PAAS|
|---|---|---|---|
|0|Where it runs?|Runs in any cloud computer or virtual machine or computer |Underlying architecture is already handled by |
|1|Who is going to Setup?|User needs to setup with packages required|Setup is already done|
|2|Customizability|Very modular & highly customizable|Less customizability|
|3|Is it easy to setup?| Setting up requires time| Does not require much time like IAAS|
|4|Can I interact & debug?|Run a server & work in jupyter notebook, helps run things interactively & debug each step|Debugging option is highly reduced. All the visuals generated needs to be saved before viewing|
|5|Examples|AWS EC2, Azure Virtual Machines|AWS Glue job, Azure Data Factory|


## 2. Component Selection and Data Pipeline Implementation 


In [ ]:
"""

Importing Packages
    Packages are categorized into groups based on their functionality & usage

"""

# Big Data Packages
import pyspark
from pyspark.sql import SparkSession


## Spark Data types
from pyspark.sql.types import (StructType, StructField, StringType, BooleanType, FloatType, TimestampType, DoubleType, DateType, IntegerType)

## Spark Functions
from pyspark.sql.functions import (col, when, desc, udf) # Selectors
from pyspark.sql.functions import (count, isnull, sum, mean, variance, stddev, percentile_approx, countDistinct) # Data Manipulation
import pyspark.sql.functions as psf

# Visualization Packages

import plotly
import plotly.express as px

import matplotlib as mpl
import matplotlib.pyplot as plt

## Map Visualization
import folium
from folium import plugins
from folium.plugins import HeatMap, HeatMapWithTime


# Data Manipulation Packages 
import pandas as pd
import numpy as np


# Modelling Packages
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


# Additional Packages
from collections import defaultdict, OrderedDict

import os
import re
from datetime import datetime

import warnings
import requests

# Styling
from IPython.display import display, HTML


In [ ]:
"""
Configuring Jupyter environment
    Ignore warnings caused by depricated package
    
"""

# Ignore warnings
warnings.filterwarnings('ignore')
requests.packages.urllib3.disable_warnings()

# Visualization Figure Size
plt.rcParams['figure.figsize'] = [18, 8] # [width, height]



In [ ]:
"""

Creating Spark Session for data processing & Configuring Spark

Spark Configuration: 

    spark.sql.repl.eagerEval.enabled -> 
        Displays spark dataframe in table (like pandas table out) instead printing


    spark.sql.execution.arrow.pyspark.enabled -> (For Spark version 3.x.x)
        Apache Arrow is an in-memory columnar data format to efficiently transfer data between JVM and Python processes. 
        This is most beneficial to users that work with Pandas/NumPy data

    spark.sql.execution.arrow.enabled -> (For Spark version 2.x.x) 
        Deprecated, and maybe removed in future. Using this incase `spark.sql.execution.arrow.enabled` fails to load
        Apache Arrow is an in-memory columnar data format to efficiently transfer data between JVM and Python processes. 
        This is most beneficial to users that work with Pandas/NumPy data


"""

# Finding Spark

import findspark
findspark.init()
findspark.find()


spark = SparkSession.builder.appName('Spark')\
                                    .config('spark.sql.repl.eagerEval.enabled',True)\
                                    .config('spark.sql.execution.arrow.enabled', True)\
                                    .config('spark.sql.execution.arrow.pyspark.enabled',True)\
                                    .config("spark.driver.memory", "15g")\
                                    .enableHiveSupport().getOrCreate()

# Getting Spark Session
spark 

## 3. Data Extraction and Filtering System running, test and diagnostics

#### Given Specifics:

2.	Filter the dataset so that only relevant crimes are included.




In [ ]:
# Custom Functions to optimize code reusability

def checkCountDifference(spark_df, name):
    '''
    prints:
        for each column in spark dataframe:
            Count of values (nulls excluded), Count of null values (#of rows in spark dataframe - count of column)
    '''
    print(f'Checking for missing values : {name}')
    for column_ in spark_df.columns:
        res = spark_df.select(count(col(column_)).alias('count'))
        count_ = res.collect()[0]['count']
        print(f'Column: {column_} | Count: {count_} | Missing: {spark_df.count()-count_}')

def renameColumns(spark_df):
    '''
    Renames Spark dataframe columns with ALL_CAPITAL & `_` separated instead of space ` `
    
    '''
    for old_column_name in spark_df.columns:
        spark_df = spark_df.withColumnRenamed(old_column_name, old_column_name.upper().replace(' ','_'))
    return spark_df


def stylePandasDataframe(df):
    '''
    Pandas dataframe styler. Prints dataframe in a grid format
    
    '''
    
    
    style = df.style.set_table_styles(
        [{"selector": "", "props": [("border", "1px solid grey")]},
          {"selector": "tbody td", "props": [("border", "1px solid grey")]},
         {"selector": "th", "props": [("border", "1px solid grey")]}
        ]
    )
    return HTML(style.render())


In [ ]:
# Custom Functions for Visualization 
# Every function below have args:dict as parameter -> which will be unpacked inside the function

def plotlyTreeMap(dataframe, args) -> None:
    '''
    Plots and displays a tree map 
    Why we are using Tree Map with 0 depth?
        This way, it helps us in visualize proportion in a sorted manner 
        
    '''
    fig = px.treemap(dataframe, color_discrete_sequence=px.colors.sequential.RdBu, color_continuous_scale='RdBu', **args)
    fig.show()

    
    
def plotlyBar(dataframe, args):
    '''
    Plots and displays interactable Bar chart
    
    '''
    fig = px.bar(dataframe, **args)
    fig.show()
    
    
def plotlyLine(dataframe, args):
    '''
    Plots and displays interactable Line chart
    
    '''
    fig = px.line(dataframe, **args)
    fig.show()
    

In [ ]:
"""

Loading Data 

"""


crimes_df = spark.read.csv(path=r'all_crimes21_hdr.txt'
                                 , header=True
                                 , inferSchema=True)

# Renaming columns to minimize errors while referencing columns 
crimes_df = renameColumns(crimes_df)



In [ ]:
'''
# Feature engineering
1. We need Area name to know events at aggregated level. 
    With use of LSOA_NAME, we can extract AREA_NAME by removing last 5 characters

2. Extract Month and Year from the Timestamp

'''
## Extracting Area name from LSOA Name
crimes_df = crimes_df.withColumn('AREA_NAME', psf.expr("substring(LSOA_NAME, 1, length(LSOA_NAME)-5)"))


## Extracting month & year from timestamp
crimes_df = crimes_df.withColumn('EXTRACTED_MONTH', psf.expr("month(MONTH)"))
crimes_df = crimes_df.withColumn('YEAR', psf.expr("year(MONTH)"))



## 4. Design, Development and reasoning behind use of multiple visualization methods, statistics, and machine learning Models

3.	Using appropriate techniques, determine whether Burglaries are increasing, decreasing, or are stable.
4.	Using appropriate techniques, determine whether Burglary is more closely associated with areas of high affluence, relative deprivation or neither
5.	Select and prepare no more than four visualizations to support your analytic findings from (3).

# General View 
## Crimes across Year-Month 

### Observation:
    * Overall crimes in UK have a cyclic pattern with peak in mid year
    * Overall crime total decreased during 2014 to 2016 comparing to rest of the years
    



In [ ]:
crimes_by_month_group = crimes_df.groupby(['MONTH', ])
all_crimes_by_month = crimes_by_month_group.count().sort('MONTH', desc('count'),)
all_crimes_by_month_df = all_crimes_by_month.toPandas() # Converting to pandas for visualization
plotlyLine(all_crimes_by_month_df, {'x':'MONTH', 'y':'count', 'title':'Fig 1. All Crimes between 2010 to 2021'})


### Observation:
* `Burglary` is in 5th position among all crimes sorted down in descending order of `#of Crimes`
* Pattern of `Shoplifting`, `Burglary`, `Vehicle crime` & `Criminal damange and arson` are similar
* `Other Crimes` initally seems to be miscategorized 
* `Violence and sexual offences` is increasing over time
* `Anti-soical behaviour` trend keeps decreasing till 2019 and suddenly increases in 2020. Probably due to outrise in corona
* Drugs have a constant trend over time
* `Violent Crime` has a declining trend. Data is not available after Apr-2013
* If `Violent Crime` and `Violence and sexual offences` are combined together, there is an upward trend. As of 2021, Crimes increased 3x times of 2013

In [ ]:
"""
Comparing crime types trend at month level

"""
crime_by_type_month_group = crimes_df.groupby(['MONTH', 'CRIME_TYPE']) 
crimes_by_month = crime_by_type_month_group.count().sort('MONTH', desc('count'),)
crimes_by_month_df = crimes_by_month.toPandas() # Converting to pandas for visualization

plotlyLine(crimes_by_month_df, {'x':'MONTH', 'y':'count'
                                , 'color':'CRIME_TYPE'
                                , 'title':'Fig 2. Trend of All Crime Types'})



# Finding Burglary Crimes From Data 

### Checking Various Crimes Types

Grouping crimes types together and taking count of crimes


#### Observations:
* Among all crimes, 6% is Burglary crimes
* Drug crimes is bigger than weapon crimes
* Anti-social behaviour is 2x of Violence and sexual offences 



In [ ]:
# Checking Various Crimes Types

"""
Grouping crimes types together and taking count of crimes

""" 

crimes_by_type_group = crimes_df.groupby('CRIME_TYPE') 
crimes_df_groupby_df = crimes_by_type_group.count().sort(desc('count'))

crime_type_df = crimes_df_groupby_df.toPandas() # Converting to pandas for visualization
crime_type_df['CRIME_%'] = round((crime_type_df['count']/crime_type_df['count'].sum()) *100, 2)

plotlyTreeMap(crime_type_df,  {'path':['CRIME_TYPE'], 'values':'count','title':'Fig 3. Proportion View Of Crimes'
                               ,'hover_name':"CRIME_TYPE"
                               , 'hover_data':['count','CRIME_%']})

plotlyBar(crime_type_df, {'x':'CRIME_TYPE', 'y':'count', 'title':'Fig 4. Number of Crimes Cases by Type'
                          ,'hover_name':"CRIME_TYPE"
                          , 'hover_data':['count','CRIME_%']})
display(crime_type_df)


# 5. Selection, application, and reasoning behind use of statistical analysis and multiple evaluation measures 

# Burglary Protection 
## Client: Insurance company developing a highly segmented home insurance product

### Claims to prove:
1. There are more burglaries in more affulent areas
2. Burglaries are increasing, decreasing, or are stable


# To prove the claims:
#### 1. There are more burglaries in more affulent areas
1. Approach: Rank the Area names based on total #of Burglary crimes over years
    ## Observations:
        1. Leeds, Birmingham, Sheffield, Bradford, Liverpool, Bristol, Manchester are top areas where the Burglary crimes
        trends are high compared to other areas
        2. We can create buckets / bins based on the ranks / #of crimes
    

#### 2. Burglaries are increasing, decreasing, or are stable
1. Approach: Look at the trend of burglaries over time
    ## Observations:
    1. Cyclic trend of Burglary crimes is consistent & peaks during end of the year.
    2. Between Jan 2014 and Mar 2019, the trend is $\pm$ 3K crimes from the mean of 35K crimes
    3. Burglary reduced by 70% comparing Jan 2020 and March 2020 due to [Covid-19 based on Timeline of UK government coronavirus lockdowns and measures, March 2020 to December 2021](https://www.instituteforgovernment.org.uk/sites/default/files/2022-12/timeline-coronavirus-lockdown-december-2021.pdf)
    ## Conclusion:
    #### 1. Forecasting of Burglary crimes are significantly skewed due to Covid-19 effect. 
    #### 2. If we negate the covid-19 period (2019 FY - 2021 FY), Burglary crimes started declining and became stable after Jan 2014.
    #### 3. Across the timeline with crime peaking during end of Year
    
    
2. Aggregate by area names and look at their trend



# Test Claim #1 " Burglary Crime is increasing, decreasing ,or is stable"
### To prove the claim, 
1. Filter for CRIME_TYPE='Burglary'
2. Look at trend over time
3. Additional insight: Look at trend over time by LSOA Area


In [ ]:

'''
Filter for Burglary Crimes

'''

Burglary_crime_df = crimes_df.filter((col('CRIME_TYPE')=='Burglary'))

Burglary_crime_by_year_area_df = Burglary_crime_df.groupby(['YEAR', 'AREA_NAME']).agg(count(col('MONTH')).alias('NUMBER_OF_CRIMES'))


# Converting to pandas
Burglary_crime_rank_df = Burglary_crime_by_year_area_df.select(*['AREA_NAME', 'YEAR', 'NUMBER_OF_CRIMES']).toPandas()

# Adding Ranks based on descending order of `NUMBER_OF_CRIMES`  
# Lower rank means highly populated with `Burglary crimes` in that Area

Burglary_crime_rank_df['BURGLARY_CRIMES_RANK'] = Burglary_crime_rank_df.groupby(['YEAR'])['NUMBER_OF_CRIMES'].rank('first', ascending=False)
Burglary_crime_rank_df['BURGLARY_CRIMES_RANK'] = Burglary_crime_rank_df['BURGLARY_CRIMES_RANK'].astype(int)



In [ ]:
print('What are the top 20 Areas involving Burglary incidents?')

print('Top 20 Burglary cases Based on BURGLARY_CRIMES_RANK')

stylePandasDataframe(pd.pivot_table(Burglary_crime_rank_df[Burglary_crime_rank_df['BURGLARY_CRIMES_RANK']<21].fillna('Unknown Area Name')
               , values=['AREA_NAME'], index=['YEAR',]
               , columns=['BURGLARY_CRIMES_RANK']
               , aggfunc='min', fill_value='', dropna=False, sort=True))


In [ ]:
Burglary_crime_df_overall = Burglary_crime_df.groupby('AREA_NAME').agg(count(col('MONTH')).alias('NUMBER_OF_CRIMES'))
Burglary_crime_df_overall_pdf = Burglary_crime_df_overall.toPandas()


# Adding Ranks based on descending order of `NUMBER_OF_CRIMES`  
# Lower rank means highly populated with `Burglary crimes` in that Area

Burglary_crime_df_overall_pdf['OVERALL_BURGLARY_CRIMES_RANK'] = Burglary_crime_df_overall_pdf['NUMBER_OF_CRIMES'].rank(method='first', ascending=False)
Burglary_crime_df_overall_pdf['OVERALL_BURGLARY_CRIMES_RANK'] = Burglary_crime_df_overall_pdf['OVERALL_BURGLARY_CRIMES_RANK'].astype(int)

Burglary_crime_df_overall_pdf.fillna('Unknown Area Name', inplace=True)

In [ ]:
print('What are the top 20 Areas involving Burglary incidents Across Years ?')

print('Top 20 Burglary cases Based on OVERALL_BURGLARY_CRIMES_RANK')

stylePandasDataframe(pd.pivot_table(Burglary_crime_df_overall_pdf[Burglary_crime_df_overall_pdf['OVERALL_BURGLARY_CRIMES_RANK']<21]
               , values=['AREA_NAME']
               , index=['OVERALL_BURGLARY_CRIMES_RANK']
               , aggfunc='min', fill_value='', dropna=False, sort=True))


In [ ]:
Burglary_crime_df_overall_pdf['Crime_Bins'] = pd.cut(Burglary_crime_df_overall_pdf['NUMBER_OF_CRIMES'], bins=10, labels=[f'Crime_Bin_{_}' for _ in range(1,11)],)

Burglary_crime_df_overall_pdf.groupby(['Crime_Bins'], sort=True).agg(number_of_areas = pd.NamedAgg(column="AREA_NAME", aggfunc="count")
    ,min_number_of_crime = pd.NamedAgg(column="NUMBER_OF_CRIMES", aggfunc="min")
    , max_number_of_crime = pd.NamedAgg(column="NUMBER_OF_CRIMES", aggfunc="max"))

In [ ]:
from IPython.display import IFrame
print("Fig 5. Income estimates for small areas, England and Wales: financial year ending 2018")

IFrame('https://www.ons.gov.uk/visualisations/dvc767/map/index.html', width='100%', height='531px')



# Observations
1. Forecasting of Burglary crimes are significantly skewed due to Covid-19 effect. 
2. If we negate the covid-19 period (2019 FY - 2021 FY), Burglary crimes started declining and became stable after Jan 2014.
3. Across the timeline with crime peaking during end of Year
    


In [ ]:
print('Claim #2. Burglaries are increasing, decreasing, or are stable')
print('Test 1: Check if Burglary Crimes is Increasing')

Burglary_crimes_by_month_df = Burglary_crime_df.groupby(['MONTH',]).count().sort('MONTH', desc('count'),)
Burglary_crimes_by_month_pdf = Burglary_crimes_by_month_df.toPandas()
print('Burglary crime count mean value: ', Burglary_crimes_by_month_pdf['count'].mean())
plotlyLine(Burglary_crimes_by_month_pdf, {'x':'MONTH', 'y':'count', 'title':'Fig 6. Claim #1. Burglary Crimes is Increasing'})


In [ ]:

print('Test 2: Check if "Burglaries are increasing, decreasing, or are stable" by LSOA Area')

Burglary_crimes_by_area_month_df = Burglary_crime_df.groupby(['MONTH','AREA_NAME']).count().sort('AREA_NAME', 'MONTH', desc('count'),)
Burglary_crimes_by_area_month_pdf = Burglary_crimes_by_area_month_df.toPandas()
plotlyLine(Burglary_crimes_by_area_month_pdf.fillna('Unknown Area Name').sort_values(['MONTH', 'count', 'AREA_NAME'], ascending=False), {'x':'MONTH', 'y':'count'
                                                       , 'animation_frame':'AREA_NAME'
                                                       , 'animation_group':'AREA_NAME'
                                                       ,'range_y':[0, 1200]
                                                       , 'title':'Fig 7. Claim #2. "Burglaries are increasing, decreasing, or are stable" By LSOA Area'})



### UK Geograpy - Burglary Density Heatmap Observations
* Crimes are more in `Counties`
* Crimes are highly dense in areas like London, Cardiff, Birmingham, Leeds, Manchester & coastal areas


In [ ]:
# Map
burglary_crimes_lat_long_crimes_df = Burglary_crime_df.groupby(['LATITUDE', 'LONGITUDE']).agg(psf.expr('count(MONTH) as BURGLARY_COUNT')).toPandas()

burglary_crimes_map = folium.Map(location=[51.4203,0.0705], tiles='stamentoner', zoom_start=6)

# Add a heatmap to the base map
HeatMap(data=burglary_crimes_lat_long_crimes_df[['LATITUDE', 'LONGITUDE','BURGLARY_COUNT']].dropna(), radius=12, max_opacity=0.3).add_to(burglary_crimes_map)

# Display the map
print('Fig 8. Burglary Density Heatmap')
burglary_crimes_map



# Using Machine Learning To Forecast Crimes
### Using Regression model to forecast  crimes related to (Violen crimes, weapon crimes & drug crimes)

In [ ]:
"""
Comparing Burglary crime types trend at month level
Filter for Burglary Crimes
"""


crimes_by_month = Burglary_crime_df.groupby(['MONTH', 'CRIME_TYPE']).count().sort('MONTH', desc('count'),)
crimes_by_month_df = crimes_by_month.toPandas() # Converting to pandas for visualization


In [ ]:

plotlyLine(crimes_by_month_df, {'x':'MONTH', 'y':'count', 'color':'CRIME_TYPE', 'title':'Fig 9. Trend of Burglary Crime'})


## 6. Detailed Analysis and consideration of the appropriateness of the solution for the initial problem

### Observations:
* **Filter:** We are narrowing down to 'Burglary' for forecasting 
* **Forecast to future:** 12 Month window 
* **Machine Learning Model:** Linear Regression 

### Model Performance:
* The regression model was able to predict as close as possible
* We can also try with various other regressor models like xgboost, catboost, lightgbm and so on.
* Another approach is to go with Time series models like fbprophet, ARIMA, SARIMA models

### Feature engineering:
* When using Regression model, we need to create lags of target variable.
    * Using lags, we can introduce seasonality in the data. 
    * This is one of the way, time series models use to create relationship behind the seasonality pattern


In [ ]:
burglary_crimes_month_df = Burglary_crime_df.groupby('MONTH').agg(count('MONTH').alias('NUMBER_OF_CRIMES')).sort(desc('MONTH'))
burglary_crimes_month_df = burglary_crimes_month_df.withColumn('TEMP', psf.lit(1))

from pyspark.sql.window import Window
windowSpec  = Window.partitionBy("TEMP").orderBy("MONTH")

## Feature Engineering 
# Creation of lags to introduce seasonality
# 
burglary_crimes_month_df = burglary_crimes_month_df.withColumn('LAG1', psf.lag('NUMBER_OF_CRIMES',1).over(windowSpec))
burglary_crimes_month_df = burglary_crimes_month_df.withColumn('LAG2', psf.lag('NUMBER_OF_CRIMES',2).over(windowSpec))
burglary_crimes_month_df = burglary_crimes_month_df.withColumn('LAG3', psf.lag('NUMBER_OF_CRIMES',3).over(windowSpec))
burglary_crimes_month_df = burglary_crimes_month_df.withColumn('LAG4', psf.lag('NUMBER_OF_CRIMES',4).over(windowSpec))
burglary_crimes_month_df = burglary_crimes_month_df.withColumn('LAG5', psf.lag('NUMBER_OF_CRIMES',5).over(windowSpec))


In [ ]:
# Dropping null values caused by lags
full_model_data_df = burglary_crimes_month_df.dropna()

# Using row number to create train test split 
full_model_data_df = full_model_data_df.withColumn('ROW_NUMBER', psf.row_number().over(windowSpec))


In [ ]:
# Using 80% of data as training and remaining 20% as testing
train_size = full_model_data_df.count()*0.8

In [ ]:
# Splitting data into train & test

train_data = full_model_data_df.filter(full_model_data_df.ROW_NUMBER<train_size)
test_data = full_model_data_df.filter(full_model_data_df.ROW_NUMBER>=train_size)


In [ ]:
'''
Observation:
    * The pattern is Number of crimes involving possession of weapons & Drugs.
    * They form a U shape trend with increase in future.

'''

plotlyLine(burglary_crimes_month_df.toPandas(), {'x':'MONTH', 'y':'NUMBER_OF_CRIMES','title':'Fig 10. Trend of `Burglary`'}) #'color':'CRIME_TYPE'})

In [ ]:
train_data_df = train_data.toPandas()
train_data_df.set_index('MONTH', inplace=True)

test_data_df = test_data.toPandas()
test_data_df.set_index('MONTH', inplace=True)

# Renaming variables & Selecting features for the model
train_data_df['ACTUAL'] = train_data_df['NUMBER_OF_CRIMES']
test_data_df['ACTUAL'] = test_data_df['NUMBER_OF_CRIMES']
features = ['LAG1', 'LAG2', 'LAG3', 'LAG4', 'LAG5']
train_data_df['TYPE'] = 'TRAIN'
test_data_df['TYPE'] = 'TEST'


In [ ]:
X_train = train_data_df[[*features, 'ACTUAL']]
y_train = X_train.pop('ACTUAL')
X_test = test_data_df[[*features, 'ACTUAL']]
y_test = X_test.pop('ACTUAL')


In [ ]:
# Train the Linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Predict the forecast for the train & test data for plotting
train_data_df['PREDICTED'] = model.predict(train_data_df[features])
test_data_df['PREDICTED'] = model.predict(test_data_df[features])


In [ ]:
# Evaluate the model using mean squared error and R-squared

'''

MSE: Mean Squared Error (Thumb Rule: Lower is better)
    Average(Square (difference between Actual and predicted))
    
RMSE: Root Mean Squared Error (Thumb Rule: Lower is better)
    MSE penalizes larger errors more severely and taking Sqrt will reduce 


R-Squared = (Thumb Rule: Closer to 1 is better)
    Proportion of variability of dependent variable explained by independent variable 


'''




mse = mean_squared_error(y_test, y_pred) 
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f'MSE: {mse:.2f}') # Lower the better
print(f'RMSE: {rmse:.2f}') # Lower the better
print(f'R-squared: {r2:.2f}') # Higher the better


In [ ]:
# Make forecasts for future time periods
future_periods = 12
future_data = test_data_df.copy()

for period in range(1, future_periods+1):
    feature_values = future_data[features].iloc[-1].values.reshape(1, -1)
    forecast = model.predict(feature_values)[0]
    
    # Appending new row and making a lag shift for next input 
    future_data = future_data.append(pd.DataFrame({
                                                    'PREDICTED': forecast
                                                   , 'LAG1': forecast
                                                   , 'LAG2': feature_values[0,0]
                                                   , 'LAG3': feature_values[0,1]
                                                   , 'LAG4': feature_values[0,2]
                                                   , 'LAG5': feature_values[0,3]
                                                   , 'TYPE':'FUTURE'}
                                                  , index=[future_data.index[-1] + pd.DateOffset(months=1)]))



In [ ]:
# Combine train, test & future forecast for plotting

model_performance = pd.concat([train_data_df, future_data]).reset_index() 
model_performance['PREDICTED_TYPE'] = 'PREDICTED_'+model_performance['TYPE']
model_performance['ERROR'] = model_performance['ACTUAL']-model_performance['PREDICTED']

fig = px.line(model_performance, x='index', y='PREDICTED',color='PREDICTED_TYPE', title='Fig 11. Linear Regression Model Prediction vs ACTUAL')
fig.add_scatter(x=model_performance['index'], y=model_performance['ACTUAL'], name='ACTUAL', mode='lines+markers', line={'dash':'dot'})
fig.show()


# Observations
1. Forecasting of Burglary crimes are significantly skewed due to Covid-19 effect. 
2. If we negate the covid-19 period (2019 FY - 2021 FY), Burglary crimes started declining and became stable after Jan 2014.
3. Looking at the model's performance on train & test, we are close with prediction & (r2 score of 0.55).
4. Since (2019 FY - 2021 FY), Burglary crimes reports have reduced heavily. Yet, the model is able to recorrect the prediction with forecasts of increasing trend.
5. If we negate Covid-19 effect, we would be expecting cases to follow same seasonal trend which the model is able to predict approximately 




In [ ]:
"""

Stopping Services / Memory Clean

"""


# spark.stop()



## 7. Evaluation and Conclusion


# Takeaway for the Insurance Company
## Conclusion:

#### 1. There are more burglaries in more affulent areas
    1. Crimes are highly dense in areas like London, Cardiff, Birmingham, Leeds, Manchester & coastal areas
    2. Leeds, Birmingham, Sheffield, Bradford, Liverpool, Bristol, Manchester are top areas where the Burglary crimes trends are high compared to other areas
    3. We can create buckets / bins based on the ranks / #of crimes
    4. Use of bucketing technique, we can create clusters containing areas which can be used to set policy price.
       #### A better approach instead of Binning is **K-Means** Segmentation. 
            1. Create various features for the model 
            2. Score the areas 
            3. Between each model score runs, track how much data points are shifted from 1 segment to another
            4. When the clusters are shifted drastically (say 10%-20% across clusters) then re-run **K-Means** model again

#### 2. Burglary increasing, decreasing, or stable
* Burglary Crimes are **stable**. 
* From 2011 to 2016, there is a downward trend and later June-2016 the trend started going back to normal 
(2014 pattern)
* Based on _trend analysis over months & year_, we can conclude that Burglary crimes are **Stable** 
* Cyclic trend of Burglary crimes is consistent & peaks during end of the year.
* Between Jan 2014 and Mar 2019, the trend is $\pm$ 3K crimes from the mean of 35K crimes
* Across the timeline with crime peaking during end of Year
* Burglary reduced by 70% comparing Jan 2020 and March 2020 due to [Covid-19 based on Timeline of UK government coronavirus lockdowns and measures, March 2020 to December 2021](https://www.instituteforgovernment.org.uk/sites/default/files/2022-12/timeline-coronavirus-lockdown-december-2021.pdf)

* Forecasting of Burglary crimes are significantly skewed due to Covid-19 effect. 
* If we negate the covid-19 period (2019 FY - 2021 FY), Burglary crimes started declining and became stable after
Jan 2014.




## 8. Scientific References and Citation


1. "Crime Detection and Prediction Using Big Data Analytics: A Case Study of Chicago" by Dong Wang, et al., published in IEEE Transactions on Big Data in 2018, https://www.jetir.org/papers/JETIR2107201.pdf

2. "Using Machine Learning to Assist Crime Prevention," 2017 6th IIAI International Congress on Advanced Applied Informatics (IIAI-AAI), Hamamatsu, Japan, 2017, pp. 1029-1030, doi: 10.1109/IIAI-AAI.2017.46.

3. Office for National Statistics (ONS), published 5 March 2020, ONS website, statistical bulletin, [Income estimates for small areas, England and Wales: financial year ending 2018](https://www.ons.gov.uk/peoplepopulationandcommunity/personalandhouseholdfinances/incomeandwealth/bulletins/smallareamodelbasedincomeestimates/financialyearending2018)


4. Office for National Statistics. (2021). Crime in England and Wales. Retrieved from https://www.ons.gov.uk/peoplepopulationandcommunity/crimeandjustice/datasets/crimeinenglandandwales

* Police.UK. (2021). Police.UK - Crime and Policing in England, Wales and Northern Ireland. Retrieved from https://www.police.uk/

5. G. Jha, L. Ahuja and A. Rana, "Criminal Behaviour Analysis and Segmentation using K-Means Clustering," 2020 8th International Conference on Reliability, Infocom Technologies and Optimization (Trends and Future Directions) (ICRITO), Noida, India, 2020, pp. 1356-1360, doi: 10.1109/ICRITO48877.2020.9197791.

6. https://www.ons.gov.uk/economy/inflationandpriceindices/timeseries/l55o/mm23

7. https://www.ons.gov.uk/economy/inflationandpriceindices/bulletins/consumerpriceinflation/march2023#:~:text=The%20CPIH%20inflation%20rate%20is,down%20from%2013.4%25%20in%20February.
 